In [1]:
# python libraries
import numpy as np
from scipy.integrate import ode
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# hacks
import os
import sys
nb_dir = os.path.split(os.path.abspath(os.getcwd()))[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

### Test Files:

In [4]:
# test files
import Dynamics

### Convienent Locations:

In [5]:
cartesian_Earth = np.array([1,0,0]) # [AU]
cartesian_Sun   = np.array([0,0,0]) # [AU]

### Convienent Directions:

In [6]:
cartesian_fromSun = np.array([1,0,0])
cartesian_toSun   = np.array([-1,0,0])
cartesian_right   = np.array([0,1,0])
cartesian_left    = np.array([0,-1,0])
cartesian_up      = np.array([0,0,1])
cartesian_down    = np.array([0,0,-1])

### Convienent Zs:

In [7]:
Z_neutron  = 0
Z_proton   = 1
Z_deuteron = 1
Z_alpha    = 2
Z_helium   = 4
Z_carbon   = 6
Z_nitrogen = 7
Z_oxygen   = 8
Z_neon     = 10
Z_argon    = 18
Z_iron     = 26
Z_krypton  = 36
Z_xenon    = 54
Z_uranium  = 92

# Trajectory

In [72]:
def solveTrajectory(cartesian_pos, cartesian_beta, Z, E):
    """Calcuates the trajectory of nuclei Z with energy E.
    Assumes ultra-relativistic nuclei with beta > 0.999990 ~= 1.
    returns an array of cartesian positions describing the path solved.
    """
    initial_s    = 0.
    initial_pos  = np.array(cartesian_pos)
    initial_beta = np.array(cartesian_beta)
    # enforce special relativity:
    initial_beta = initial_beta / np.sqrt( np.dot(initial_beta, initial_beta) )
    initial_conditions = np.concatenate(( initial_pos, initial_beta ))
    
    ratio = Z / float(E)

    ### TODO: try different integrators
    integrator = ode(Dynamics.applyForces).set_integrator('dopri5', nsteps=100000000)
    integrator.set_initial_value(initial_conditions, initial_s).set_f_params(ratio)

    AU2m = 149597870700. # use:  number [AU] * AU2m = converted number [m]
    final_s = 5 * AU2m   # track particle for 5 AU (in meters)
    ds = 1e-6 * AU2m     # numerical stepsize, ds distance (in meters)
    positions = []
    iterations = 0
    limit = 100000
    while (integrator.successful() and integrator.t < final_s) and (iterations < limit):
        integrator.integrate( integrator.t + ds ) # because "dt" is "ds" in this context
        positions.append(integrator.y[:3])
        iterations += 1
    print integrator.get_return_code()
    return np.array(positions)

In [73]:
positions = solveTrajectory(cartesian_Earth, cartesian_toSun, Z_uranium, 2e-18)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(positions[:,0], positions[:,1], positions[:,2])

KeyboardInterrupt: 